In [59]:
import pandas as pd
from datetime import datetime
from tqdm import tqdm

In [66]:
csv_filename = '../temp.csv'
groupby_time_frequency = 'H' # H indicates hourly
aggregate_csv_filename = '../agg_temp.csv'

In [63]:
df = pd.read_csv(csv_filename)
df['completion_datetime'] = pd.to_datetime(df.complete_timestamp, format='%Y-%m-%dT%H:%M:%S.%f+00:00')
df.drop(columns='complete_timestamp', inplace=True)
df.fillna('None', inplace=True)
df.head()

,case_id,activity,resource,completion_datetime
0,2182,Start Event,None,2019-05-24 11:04:37.943
1,2182,Task 1,Resource 1,2019-05-24 11:10:55.750
2,2182,Task 2,Resource 2,2019-05-27 09:05:39.287
3,2182,Task 3,Resource 1,2019-05-27 09:09:58.065
4,2182,End Event,None,2019-05-27 09:09:58.065


In [64]:
aggregate_df = pd.DataFrame(columns=['timestamp', 'activity', 'resource', 'count'])

grouped = df.groupby(pd.Grouper(key='completion_datetime', freq=groupby_time_frequency))
for name1, group1 in tqdm(grouped):
    for name2, group2 in group1.groupby(['activity', 'resource']):
        aggregate_df = aggregate_df.append({
                'timestamp': name1,
                'activity': name2[0],
                'resource': name2[1],
                'count': len(group2['case_id'].unique())
            }, ignore_index=True)

aggregate_df.head()

100%|██████████| 7064/7064 [00:37<00:00, 190.68it/s]


,timestamp,activity,resource,count
0,2019-01-16 09:00:00,End Event,None,2
1,2019-01-16 09:00:00,Start Event,None,4
2,2019-01-16 09:00:00,Task 1,None,1
3,2019-01-16 09:00:00,Task 1,Resource 1,3
4,2019-01-16 09:00:00,Task 2,Resource 2,2


In [67]:
aggregate_df.to_csv(aggregate_csv_filename, index=False)